In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!pip install geopy
import json # library to handle JSON files


#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


In [5]:
!pip install bs4
import numpy as np 
import pandas as pd 
from bs4 import BeautifulSoup
import requests

     |████████████████████████████████| 122kB 4.6MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [7]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
can_html  = BeautifulSoup(source, 'xml')
can_html = can_html.find('table')

In [16]:
row_data = []
for tr_cell in can_html.find_all('tr'):
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
        #print(td_cell.text.strip())
print(len(row_data))
row_data[:10]

180


['M1ANot assigned',
 'M2ANot assigned',
 'M3ANorth York(Parkwoods)',
 'M4ANorth York(Victoria Village)',
 'M5ADowntown Toronto(Regent Park / Harbourfront)',
 'M6ANorth York(Lawrence Manor / Lawrence Heights)',
 "M7AQueen's Park(Ontario Provincial Government)",
 'M8ANot assigned',
 'M9AEtobicoke(Islington Avenue)',
 'M1BScarborough(Malvern / Rouge)']

In [17]:
#extract data into a list
borough  = [row_data[i][3:] for i in range(len(row_data))]
postalCode = [row_data[i][:3] for i in range(len(row_data))]
print(postalCode[:5])
print(borough[:5])

['M1A', 'M2A', 'M3A', 'M4A', 'M5A']
['Not assigned', 'Not assigned', 'North York(Parkwoods)', 'North York(Victoria Village)', 'Downtown Toronto(Regent Park / Harbourfront)']


In [18]:
# make df 
can_df = pd.DataFrame()
can_df['Postalcode'] = postalCode
can_df['temp'] = borough
can_df.head()

,Postalcode,temp
0,M1A,Not assigned
1,M2A,Not assigned
2,M3A,North York(Parkwoods)
3,M4A,North York(Victoria Village)
4,M5A,Downtown Toronto(Regent Park / Harbourfront)


In [19]:
can_df = can_df[can_df.temp != "Not assigned"] # delect df row which are 'Not assigned'
can_df[['Borough','Neighborhood','na']] = can_df.temp.str.split("(",expand = True,) # split col:temp
can_df['Neighborhood'] = can_df['Neighborhood'].str.replace(')','') 
can_df['Neighborhood'] = can_df['Neighborhood'].str.replace('/',',')
can_df = can_df.drop(['temp','na'],axis=1) # drop columns
can_df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Queen's Park,Ontario Provincial Government


In [21]:
print(can_df.shape)

(103, 3)


In [23]:
#import lat long data from csv file
latlong = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')

latlong.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [26]:
#join dataframes
can_df = can_df.join(latlong.set_index('Postal Code'), on='Postalcode')
can_df.head(10)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
6,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
8,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
9,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
11,M3B,North York,Don MillsNorth,43.745906,-79.352188
12,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
